In [1]:
import numpy as np
import pandas as pd
from electrcity_processing_functions import process_electricity_df_semi_annually, tax_tier_map, \
    transform_elec_price_to_tax_tier_time_series

#need to combine both dataframes into one semiannual frequency and check if I can model it as an AR maybe
#The problem is I have different classes, I could try doing something like fama-mcbeth maybe

elec_hh_pre_2007 = pd.read_csv('./ggdp_data/eurostat_electricity/eurostat_electricity_price_hh_pre_2007.csv', header=0)
fin_hh_pre_2007_time_series = transform_elec_price_to_tax_tier_time_series(elec_hh_pre_2007, country_code='FI',
                                                                           currency='EUR')
#Denmark
dk_hh_pre_2007_time_series = transform_elec_price_to_tax_tier_time_series(elec_hh_pre_2007, country_code='DK',
                                                                          currency='EUR')


def get_country_code_tax_tier_format(tax_tier_country_code_format: str):
    tax_tier, country_code = tax_tier_country_code_format.split('_')
    tax_tier = tax_tier.replace('.0', '')
    return f"{country_code}_{tax_tier}"


dk_fin_merged = fin_hh_pre_2007_time_series.join(dk_hh_pre_2007_time_series, how='right', lsuffix='_fi', rsuffix='_dk')
dk_fin_merged = dk_fin_merged.rename(columns=get_country_code_tax_tier_format)


C:\Users\yarde\PycharmProjects\ggdp_research\electrcity_processing_functions.py:74: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  elec_df_semi_annualy = elec_df_semi_annualy.replace({'tax_tier': tax_tier_map})


KeyboardInterrupt: 

In [2]:
import statsmodels.api as sm
from IPython.display import display

from data_reconstruction_functions import RegressionData

tax_tiers = [str(tax_tier).replace('.0', '') for tax_tier in fin_hh_pre_2007_time_series.columns.values]

models_results = []

for curr_tax_tier in tax_tiers:
    current_df = dk_fin_merged[[f'fi_{curr_tax_tier}', f'dk_{curr_tax_tier}']]
    current_df = current_df.dropna(how='any', axis=0)
    curr_formula = f'fi_{curr_tax_tier} ~ dk_{curr_tax_tier}'
    model = sm.OLS.from_formula(curr_formula, data=current_df)
    results = model.fit()
    reg_data = RegressionData(y_name=f"fi_{curr_tax_tier}", x_name=f"dk_{curr_tax_tier}", reg_result=results)
    models_results.append(reg_data)
    display(results.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 fi_600   R-squared:                       0.525
Model:                            OLS   Adj. R-squared:                  0.505
Method:                 Least Squares   F-statistic:                     26.55
Date:                Thu, 23 May 2024   Prob (F-statistic):           2.82e-05
Time:                        00:13:11   Log-Likelihood:                 74.785
No. Observations:                  26   AIC:                            -145.6
Df Residuals:                      24   BIC:                            -143.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0728      0.020      3.610      0.001       0.031       0.114
dk_600         0.3368      0.065      5.153      0.000       0.202       0.472
==============================================================================
Omnibus:                        5.558   Durbin-Watson:                   0.199
Prob(Omnibus):                  0.062   Jarque-Bera (JB):                3.028
Skew:                          -0.611   Prob(JB):                        0.220
Kurtosis:                       1.858   Cond. No.                         25.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                fi_1200   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.552
Method:                 Least Squares   F-statistic:                     31.75
Date:                Thu, 23 May 2024   Prob (F-statistic):           8.42e-06
Time:                        00:13:11   Log-Likelihood:                 87.204
No. Observations:                  26   AIC:                            -170.4
Df Residuals:                      24   BIC:                            -167.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0594      0.012      4.997      0.000       0.035       0.084
dk_1200        0.2721      0.048      5.635      0.000       0.172       0.372
==============================================================================
Omnibus:                        7.857   Durbin-Watson:                   0.225
Prob(Omnibus):                  0.020   Jarque-Bera (JB):                2.965
Skew:                          -0.506   Prob(JB):                        0.227
Kurtosis:                       1.691   Cond. No.                         29.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                fi_3500   R-squared:                       0.577
Model:                            OLS   Adj. R-squared:                  0.559
Method:                 Least Squares   F-statistic:                     32.71
Date:                Thu, 23 May 2024   Prob (F-statistic):           6.83e-06
Time:                        00:13:11   Log-Likelihood:                 96.839
No. Observations:                  26   AIC:                            -189.7
Df Residuals:                      24   BIC:                            -187.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0532      0.008      6.831      0.000       0.037       0.069
dk_3500        0.2167      0.038      5.719      0.000       0.139       0.295
==============================================================================
Omnibus:                        5.857   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.053   Jarque-Bera (JB):                2.892
Skew:                          -0.563   Prob(JB):                        0.236
Kurtosis:                       1.816   Cond. No.                         33.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                fi_7500   R-squared:                       0.515
Model:                            OLS   Adj. R-squared:                  0.495
Method:                 Least Squares   F-statistic:                     25.46
Date:                Thu, 23 May 2024   Prob (F-statistic):           3.70e-05
Time:                        00:13:11   Log-Likelihood:                 100.02
No. Observations:                  26   AIC:                            -196.0
Df Residuals:                      24   BIC:                            -193.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0492      0.007      7.289      0.000       0.035       0.063
dk_7500        0.1762      0.035      5.046      0.000       0.104       0.248
==============================================================================
Omnibus:                        5.059   Durbin-Watson:                   0.283
Prob(Omnibus):                  0.080   Jarque-Bera (JB):                2.190
Skew:                          -0.388   Prob(JB):                        0.335
Kurtosis:                       1.808   Cond. No.                         34.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 fi_inf   R-squared:                       0.811
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     102.9
Date:                Thu, 23 May 2024   Prob (F-statistic):           3.70e-10
Time:                        00:13:11   Log-Likelihood:                 109.61
No. Observations:                  26   AIC:                            -215.2
Df Residuals:                      24   BIC:                            -212.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0210      0.004      5.277      0.000       0.013       0.029
dk_inf         0.2230      0.022     10.146      0.000       0.178       0.268
==============================================================================
Omnibus:                        3.000   Durbin-Watson:                   0.300
Prob(Omnibus):                  0.223   Jarque-Bera (JB):                2.055
Skew:                          -0.500   Prob(JB):                        0.358
Kurtosis:                       2.052   Cond. No.                         31.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
from data_reconstruction_functions import fill_missing_values

fi_hh_pre_2007_recovered = dk_fin_merged.copy()

column_names = {str(col).replace('.0', '') for col in fin_hh_pre_2007_time_series}

# missing_values_map = {f"fi_{col}": f"dk_{col}" for col in column_names}

insignificant_models = []
curr_reg: RegressionData
for curr_model in models_results:
    result = fill_missing_values(fi_hh_pre_2007_recovered, y_name=curr_model.y_name, x_name=curr_model.x_name,
                                 reg_result=curr_model.reg_result)
    if not result:
        insignificant_models.append(curr_model.y_name)
print(insignificant_models)

finland_column_names = fi_hh_pre_2007_recovered.columns.values
col: str
finland_column_names = [col for col in fi_hh_pre_2007_recovered.columns.values if col.startswith('fi')]
fi_hh_pre_2007_recovered = fi_hh_pre_2007_recovered[finland_column_names]
fi_hh_pre_2007_recovered = fi_hh_pre_2007_recovered.rename(columns=lambda x: x.replace('fi_', ''))
fi_hh_pre_2007_recovered['year'] = fi_hh_pre_2007_recovered.index


fi_hh_pre_2007_recovered.to_csv('ggdp_data/imputated_data/fi_hh_pre_2007_recovered.csv', index=False)

C:\Users\yarde\PycharmProjects\ggdp_research\data_reconstruction_functions.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if reg_result.pvalues[0] > sig_level or reg_result.pvalues[1] > sig_level:
C:\Users\yarde\PycharmProjects\ggdp_research\data_reconstruction_functions.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  reg_function = lambda x: reg_result.params[0] + x * reg_result.params[1]


[]


KeyboardInterrupt: 

In [15]:
from dataclasses import dataclass

elec_nh_pre_2007 = pd.read_csv('./ggdp_data/eurostat_electricity/eurostat_electricity_price_nh_pre_2007.csv', header=0)
fin_nh_pre_2007_time_series = transform_elec_price_to_tax_tier_time_series(elec_nh_pre_2007, country_code='FI',
                                                                           currency='EUR')
#Denmark
dk_nh_pre_2007_time_series = transform_elec_price_to_tax_tier_time_series(elec_nh_pre_2007, country_code='DK',
                                                                          currency='EUR')
dk_tier_diff = pd.DataFrame()
numbers_to_tiers_map = {}
for i in range(0, len(dk_nh_pre_2007_time_series.columns)):
    numbers_to_tiers_map[str(i)] = dk_nh_pre_2007_time_series.columns.values[i]
for i in range(0, len(dk_nh_pre_2007_time_series.columns) - 1):
    dk_tier_diff[f'diff_{i + 1}_{i}'] = dk_nh_pre_2007_time_series.iloc[:, i + 1] - dk_nh_pre_2007_time_series.iloc[:,
                                                                                    i]

#looking for best predictor series to be able to fill in missing data
series_with_missing_data = ["5_4", "6_5", "7_6", "8_7"]
series_with_all_data = ['1_0', '2_1', '3_2', '4_3']


@dataclass
class RegressionResult():
    r_squared = 0
    results = None
    regressor = None


best_regressors_dict = {}
for y in series_with_missing_data:
    highest_r_squared = 0
    best_regressor = None
    regression_res = RegressionResult()
    for x in series_with_all_data:
        joint_df = dk_tier_diff[[f'diff_{y}', f'diff_{x}']]
        joint_df = joint_df.dropna(axis=0, how='any')
        model = sm.OLS.from_formula(f'diff_{y} ~ diff_{x}', data=joint_df)
        curr_results = model.fit()
        if curr_results.rsquared > regression_res.r_squared:
            regression_res.r_squared = curr_results.rsquared
            regression_res.results = curr_results
            regression_res.regressor = x
    best_regressors_dict[y] = regression_res

for var, reg_result in best_regressors_dict.items():
    display(reg_result.results.summary())

C:\Users\yarde\PycharmProjects\ggdp_research\electrcity_processing_functions.py:70: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  elec_df_semi_annualy = elec_df_semi_annualy.replace({'tax_tier': tax_tier_map})
C:\Users\yarde\PycharmProjects\ggdp_research\electrcity_processing_functions.py:70: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  elec_df_semi_annualy = elec_df_semi_annualy.replace({'tax_tier': tax_tier_map})


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               diff_5_4   R-squared:                       0.426
Model:                            OLS   Adj. R-squared:                  0.400
Method:                 Least Squares   F-statistic:                     16.31
Date:                Fri, 17 May 2024   Prob (F-statistic):           0.000550
Time:                        17:29:40   Log-Likelihood:                 135.81
No. Observations:                  24   AIC:                            -267.6
Df Residuals:                      22   BIC:                            -265.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0015      0.000     -6.154      0.000      -0.002      -0.001
diff_4_3      -0.2402      0.059     -4.038      0.001      -0.364      -0.117
==============================================================================
Omnibus:                       12.546   Durbin-Watson:                   1.005
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               12.793
Skew:                           1.163   Prob(JB):                      0.00167
Kurtosis:                       5.716   Cond. No.                         331.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               diff_6_5   R-squared:                       0.450
Model:                            OLS   Adj. R-squared:                  0.425
Method:                 Least Squares   F-statistic:                     17.99
Date:                Fri, 17 May 2024   Prob (F-statistic):           0.000334
Time:                        17:29:40   Log-Likelihood:                 137.04
No. Observations:                  24   AIC:                            -270.1
Df Residuals:                      22   BIC:                            -267.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0075      0.001     -8.507      0.000      -0.009      -0.006
diff_3_2      -1.1157      0.263     -4.242      0.000      -1.661      -0.570
==============================================================================
Omnibus:                        3.389   Durbin-Watson:                   1.094
Prob(Omnibus):                  0.184   Jarque-Bera (JB):                2.415
Skew:                          -0.777   Prob(JB):                        0.299
Kurtosis:                       2.989   Cond. No.                     1.54e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.54e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               diff_7_6   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     172.7
Date:                Fri, 17 May 2024   Prob (F-statistic):           6.80e-12
Time:                        17:29:40   Log-Likelihood:                 146.63
No. Observations:                  24   AIC:                            -289.3
Df Residuals:                      22   BIC:                            -286.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0019      0.000    -11.740      0.000      -0.002      -0.002
diff_4_3      -0.4981      0.038    -13.142      0.000      -0.577      -0.419
==============================================================================
Omnibus:                        1.576   Durbin-Watson:                   0.423
Prob(Omnibus):                  0.455   Jarque-Bera (JB):                1.214
Skew:                          -0.533   Prob(JB):                        0.545
Kurtosis:                       2.722   Cond. No.                         331.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               diff_8_7   R-squared:                       0.955
Model:                            OLS   Adj. R-squared:                  0.953
Method:                 Least Squares   F-statistic:                     464.9
Date:                Fri, 17 May 2024   Prob (F-statistic):           2.76e-16
Time:                        17:29:40   Log-Likelihood:                 159.28
No. Observations:                  24   AIC:                            -314.6
Df Residuals:                      22   BIC:                            -312.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0009   9.43e-05     -9.445      0.000      -0.001      -0.001
diff_4_3       0.4825      0.022     21.560      0.000       0.436       0.529
==============================================================================
Omnibus:                       12.115   Durbin-Watson:                   1.627
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               12.096
Skew:                           1.133   Prob(JB):                      0.00236
Kurtosis:                       5.638   Cond. No.                         331.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
from typing import Callable
import numpy as np
from electrcity_processing_functions import tax_tier_map_pre_2007_nh

diff_df = dk_tier_diff.copy()
diff_df = diff_df.rename(columns=lambda x: x.replace('diff_', ''))


#completing the missing data based on these regressions.

def fill_missing_diffs(y_name: str, x_name: str, diff_df: pd.DataFrame, reg_function: Callable):
    for index, row in diff_df[diff_df[y_name].isna()].iterrows():
        # Apply the regression function to the corresponding x_name value
        diff_df.loc[index, y_name] = reg_function(row[x_name])


vars_list = list(best_regressors_dict.keys())
vars_list.sort()
for var in vars_list:
    reg_result = best_regressors_dict[var].results
    reg_function = lambda x: reg_result.params[0] + x * reg_result.params[1]
    fill_missing_diffs(y_name=var, x_name=best_regressors_dict[var].regressor, diff_df=diff_df,
                       reg_function=reg_function)

#now dynamically retrieve prices based on differences
diff_df['0'] = dk_nh_pre_2007_time_series.iloc[:, 0]
for i in range(0, len(diff_df.columns) - 1):
    current_diff = f"{i + 1}_{i}"
    diff_df.loc[:, current_diff] = diff_df.loc[:, current_diff] + diff_df[str(i)]
    diff_df = diff_df.rename(columns={current_diff: str(i + 1)})

# numbers_to_tax_tier_map = {}
# nh_tax_tiers = list(tax_tier_map_pre_2007_nh.keys())
# for i, curr_tier in enumerate(tax_tiers):
#     numbers_to_tax_tier_map[str(i)] = curr_tier

diff_df = diff_df.rename(columns=numbers_to_tiers_map)

#all values should be 0 except those that were predicted that should be NaN
test_df = diff_df - dk_nh_pre_2007_time_series



C:\Users\yarde\AppData\Local\Temp\ipykernel_11772\2268347422.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  reg_function = lambda x: reg_result.params[0] + x * reg_result.params[1]


In [17]:
#now we'll try to regress the finish elec prices on the danish elec prices (will the filled NaNs)

dk_fin_merged = fin_nh_pre_2007_time_series.join(diff_df, how='right', lsuffix='_fi', rsuffix='_dk')
dk_fin_merged = dk_fin_merged.rename(columns=get_country_code_tax_tier_format)

tax_tiers = [str(tax_tier).replace('.0', '') for tax_tier in fin_nh_pre_2007_time_series.columns.values]

reg_models_list = []

for curr_tax_tier in tax_tiers:
    curr_formula = f'fi_{curr_tax_tier} ~ dk_{curr_tax_tier}'
    curr_df = dk_fin_merged[[f'fi_{curr_tax_tier}', f'dk_{curr_tax_tier}']].dropna(how='any', axis=0)
    model = sm.OLS.from_formula(curr_formula, data=curr_df)
    results = model.fit()
    display(results.summary())
    reg_models_list.append(
        RegressionData(y_name=f"fi_{curr_tax_tier}", x_name=f"dk_{curr_tax_tier}", reg_result=results))

#filtering out Denmark data
#to fill in

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               fi_30000   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.223
Method:                 Least Squares   F-statistic:                     7.886
Date:                Fri, 17 May 2024   Prob (F-statistic):            0.00998
Time:                        17:29:42   Log-Likelihood:                 102.12
No. Observations:                  25   AIC:                            -200.2
Df Residuals:                      23   BIC:                            -197.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0628      0.005     12.273      0.000       0.052       0.073
dk_30000       0.1317      0.047      2.808      0.010       0.035       0.229
==============================================================================
Omnibus:                        3.827   Durbin-Watson:                   0.396
Prob(Omnibus):                  0.148   Jarque-Bera (JB):                1.601
Skew:                          -0.210   Prob(JB):                        0.449
Kurtosis:                       1.833   Cond. No.                         55.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               fi_50000   R-squared:                       0.369
Model:                            OLS   Adj. R-squared:                  0.342
Method:                 Least Squares   F-statistic:                     13.47
Date:                Fri, 17 May 2024   Prob (F-statistic):            0.00127
Time:                        17:29:42   Log-Likelihood:                 97.348
No. Observations:                  25   AIC:                            -190.7
Df Residuals:                      23   BIC:                            -188.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0564      0.006      8.956      0.000       0.043       0.069
dk_50000       0.2148      0.059      3.671      0.001       0.094       0.336
==============================================================================
Omnibus:                        2.894   Durbin-Watson:                   0.367
Prob(Omnibus):                  0.235   Jarque-Bera (JB):                1.518
Skew:                          -0.281   Prob(JB):                        0.468
Kurtosis:                       1.932   Cond. No.                         57.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              fi_160000   R-squared:                       0.376
Model:                            OLS   Adj. R-squared:                  0.349
Method:                 Least Squares   F-statistic:                     13.87
Date:                Fri, 17 May 2024   Prob (F-statistic):            0.00111
Time:                        17:29:42   Log-Likelihood:                 95.753
No. Observations:                  25   AIC:                            -187.5
Df Residuals:                      23   BIC:                            -185.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0485      0.007      6.847      0.000       0.034       0.063
dk_160000      0.2528      0.068      3.724      0.001       0.112       0.393
==============================================================================
Omnibus:                        5.559   Durbin-Watson:                   0.313
Prob(Omnibus):                  0.062   Jarque-Bera (JB):                1.953
Skew:                          -0.260   Prob(JB):                        0.377
Kurtosis:                       1.733   Cond. No.                         62.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             fi_1250000   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     17.32
Date:                Fri, 17 May 2024   Prob (F-statistic):           0.000376
Time:                        17:29:42   Log-Likelihood:                 93.161
No. Observations:                  25   AIC:                            -182.3
Df Residuals:                      23   BIC:                            -179.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0346      0.008      4.422      0.000       0.018       0.051
dk_1250000     0.3242      0.078      4.162      0.000       0.163       0.485
==============================================================================
Omnibus:                        6.088   Durbin-Watson:                   0.500
Prob(Omnibus):                  0.048   Jarque-Bera (JB):                2.079
Skew:                          -0.288   Prob(JB):                        0.354
Kurtosis:                       1.710   Cond. No.                         64.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             fi_2000000   R-squared:                       0.534
Model:                            OLS   Adj. R-squared:                  0.513
Method:                 Least Squares   F-statistic:                     26.33
Date:                Fri, 17 May 2024   Prob (F-statistic):           3.37e-05
Time:                        17:29:42   Log-Likelihood:                 94.385
No. Observations:                  25   AIC:                            -184.8
Df Residuals:                      23   BIC:                            -182.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0219      0.007      2.941      0.007       0.007       0.037
dk_2000000     0.3840      0.075      5.131      0.000       0.229       0.539
==============================================================================
Omnibus:                        4.728   Durbin-Watson:                   0.612
Prob(Omnibus):                  0.094   Jarque-Bera (JB):                1.723
Skew:                          -0.187   Prob(JB):                        0.423
Kurtosis:                       1.770   Cond. No.                         65.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            fi_10000000   R-squared:                       0.556
Model:                            OLS   Adj. R-squared:                  0.536
Method:                 Least Squares   F-statistic:                     27.56
Date:                Fri, 17 May 2024   Prob (F-statistic):           2.88e-05
Time:                        17:29:42   Log-Likelihood:                 90.380
No. Observations:                  24   AIC:                            -176.8
Df Residuals:                      22   BIC:                            -174.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0199      0.008      2.594      0.017       0.004       0.036
dk_10000000     0.4064      0.077      5.250      0.000       0.246       0.567
==============================================================================
Omnibus:                        5.387   Durbin-Watson:                   0.597
Prob(Omnibus):                  0.068   Jarque-Bera (JB):                1.851
Skew:                          -0.229   Prob(JB):                        0.396
Kurtosis:                       1.719   Cond. No.                         65.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            fi_24000000   R-squared:                       0.466
Model:                            OLS   Adj. R-squared:                  0.442
Method:                 Least Squares   F-statistic:                     19.23
Date:                Fri, 17 May 2024   Prob (F-statistic):           0.000236
Time:                        17:29:42   Log-Likelihood:                 87.475
No. Observations:                  24   AIC:                            -170.9
Df Residuals:                      22   BIC:                            -168.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0205      0.008      2.576      0.017       0.004       0.037
dk_24000000     0.3637      0.083      4.385      0.000       0.192       0.536
==============================================================================
Omnibus:                        5.739   Durbin-Watson:                   0.468
Prob(Omnibus):                  0.057   Jarque-Bera (JB):                1.745
Skew:                           0.072   Prob(JB):                        0.418
Kurtosis:                       1.687   Cond. No.                         62.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            fi_50000000   R-squared:                       0.314
Model:                            OLS   Adj. R-squared:                  0.283
Method:                 Least Squares   F-statistic:                     10.08
Date:                Fri, 17 May 2024   Prob (F-statistic):            0.00438
Time:                        17:29:43   Log-Likelihood:                 86.285
No. Observations:                  24   AIC:                            -168.6
Df Residuals:                      22   BIC:                            -166.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0219      0.008      2.657      0.014       0.005       0.039
dk_50000000     0.2766      0.087      3.175      0.004       0.096       0.457
==============================================================================
Omnibus:                        9.014   Durbin-Watson:                   0.431
Prob(Omnibus):                  0.011   Jarque-Bera (JB):                2.128
Skew:                          -0.114   Prob(JB):                        0.345
Kurtosis:                       1.559   Cond. No.                         62.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

C:\Users\yarde\anaconda3\envs\ggdp-research\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 fi_inf   R-squared:                       0.438
Model:                            OLS   Adj. R-squared:                  0.402
Method:                 Least Squares   F-statistic:                     12.45
Date:                Fri, 17 May 2024   Prob (F-statistic):            0.00279
Time:                        17:29:43   Log-Likelihood:                 64.235
No. Observations:                  18   AIC:                            -124.5
Df Residuals:                      16   BIC:                            -122.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0061      0.012      0.523      0.608      -0.019       0.031
dk_inf         0.4159      0.118      3.528      0.003       0.166       0.666
==============================================================================
Omnibus:                        5.410   Durbin-Watson:                   0.479
Prob(Omnibus):                  0.067   Jarque-Bera (JB):                1.537
Skew:                           0.051   Prob(JB):                        0.464
Kurtosis:                       1.572   Cond. No.                         69.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
insignificant_models = []
fi_nh_pre_2007_recovered = dk_fin_merged.copy()

curr_reg: RegressionData
for curr_model in reg_models_list:
    result = fill_missing_values(fi_nh_pre_2007_recovered, y_name=curr_model.y_name, x_name=curr_model.x_name,
                                 reg_result=curr_model.reg_result)
    if not result:
        insignificant_models.append(curr_model.y_name)

col: str
finland_column_names = [col for col in fi_nh_pre_2007_recovered.columns.values if col.startswith('fi')]
fi_nh_pre_2007_recovered = fi_nh_pre_2007_recovered[finland_column_names]
fi_nh_pre_2007_recovered = fi_nh_pre_2007_recovered.rename(columns=lambda x: x.replace('fi_', ''))
fi_nh_pre_2007_recovered['year'] = fi_nh_pre_2007_recovered.index



fi_nh_pre_2007_recovered.to_csv('ggdp_data/imputated_data/fi_nh_pre_2007_recovered.csv', index=False)


C:\Users\yarde\PycharmProjects\ggdp_research\data_reconstruction_functions.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if reg_result.pvalues[0] > sig_level or reg_result.pvalues[1] > sig_level:
C:\Users\yarde\PycharmProjects\ggdp_research\data_reconstruction_functions.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  reg_function = lambda x: reg_result.params[0] + x * reg_result.params[1]
